# signal analysis notebook(Linux)
このノートブックでは、実際に筆者が書いた関数を呼び出し、それに基づいて信号の可視化を解析・解説しています。ソースコードは同じレポジトリ（src/utils.py）などを参照してください。
このファイルは、Linux向けに動作テストを実施しています。Windows環境では、別ファイルを参照してください。

## 環境設定  
下のセルを実行し、必要なライブラリをすべてインストールしましょう。GPUがきちんと認識されているか、Pythonで使用できる状態になっているか確認してください。  
なお、今回のtorchのインストールに関しては、バージョンを指定する方法はうまく行きません。公式の対応に任せていますが、環境によっては別途での対応が求められるかもしれません。

In [ ]:
!pip install -r requirements.txt
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

## メタデータ表示
生のファイルの中にどのような情報があるのかを出力します。特に、これらはメタデータ表示用として利用してください。実際にデータが取得された年度において処理が異なることに留意してください。

In [ ]:
from src import analyze_mat_file,analyze_mat_file_h5py
file_path_exp = "/home/smatsubara/documents/airlift/data/experiments/rawsignal/P20240726-1600.mat"
file_path_sim = "/home/smatsubara/documents/airlift/data/simulation/rawsignal/solid_liquid2.mat"
analyze_mat_file(file_path_exp)
analyze_mat_file_h5py(file_path_sim)

## ファイル形式変換

実機・シミュレーションともに`.mat`形式で保存されていますが、これらはサンプリングレートや単位が異なるほか、実機において15000回の測定が連続的に記録されており、境目を表示することが困難です。  
ゆえに、処理の統一化を目的としてすべてを`.npz`形式に変換します。これにより一つのスクリプトで統一的に信号波形を処理することができるようになります。また、機械学習用のデータセットを作成しやすくなります。  
以下のセルは、実機データを変換するスクリプトです。   
### 実機データ
命名規則:
 - 2024: dates and time  
    e.g. P20241007-1013.mat
 - 2023: experimental settings(requires details)  
    e.g. s0_g2_l2_t1.mat

これらに対し、`.npz`形式に変換したものを`*_processed.npz`として新しい名前を与え保存しています。
なお、`signal_key="TDX1"`と書いてありますが、これはTDX1のみを処理しているのではなく、これらをベースにトリガを検出し、その時刻を基準点としてすべてのチャンネルの信号波形を並び変えているということに注意してください。



In [6]:
from src import mat2npz_exp


file_path = "/home/smatsubara/documents/airlift/data/experiments/rawsignal/P20241007-1013.mat"
output_dir = "/home/smatsubara/documents/airlift/data/experiments/processed"  # Change to your desired directory


# 関数を呼び出して実行
mat2npz_exp(
    file_path=file_path,
    output_dir=output_dir,
    start_time=0.0,
    duration=5.0,
    amplitude_threshold=2,
    window_width=0.1e-3,
    signal_key="TDX1"
)

Loading data...
Loading successful
Using device: cuda
Number of detected triggers: (14988,)
arranged_pulses_tdx1.shape: (14988, 5208)
arranged_pulses_tdx1_enlarged.shape: (14988, 5208)
arranged_pulses.shape: (14988, 5208, 4)
['__header__', '__version__', '__globals__', 'Tstart', 'Tinterval', 'ExtraSamples', 'RequestedLength', 'Length', 'Version', 'TDX1', 'TDX2', 'TDX3', 'TDX1_enlarged']
signal points: (5208,)
Processed data and metadata saved to: /home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz


'/home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz'

### シミュレーションデータ  
次に、シミュレーションデータを変換します。シミュレーションにおいては、計算時間の制約で複数パルス分のデータを用意することはできません。さらに、ソフトの設定によりkgridの中にdtは具体的な値としては保存されません。  
それゆえ、`config.json`をもとに復元する必要があります。そのため、`.json`ファイルも読み込んで同時に処理します。そこに、メタデータも含ませてよりリッチな情報を提供します。  
また`.npz`のファイルには１パルス分の情報だけが記録されることになります。それゆえ、それらを画像として可視化する場合、`full=True`と`full=False`の処理は同じになります。  
以下のセルはシミュレーションデータを変換するスクリプトです。


In [7]:
from src import mat2npz_sim
file_path_ex = "/home/smatsubara/documents/airlift/data/simulation/rawsignal/solid_liquid2.mat"
output_dir_ex = "/home/smatsubara/documents/airlift/data/simulation/processed"  # Change to your desired directory
config_path_ex="/home/smatsubara/documents/airlift/data/simulation/config.json"


mat2npz_sim(
    file_path=file_path_ex,
    config_path=config_path_ex,
    output_dir=output_dir_ex
)

<KeysViewHDF5 ['#refs#', '#subsystem#', 'kgrid', 'sensor_data']>
['Nt', 'Nx', 'Ny', 'Nz', 'dim', 'dt', 'dx', 'dxudxn', 'dxudxn_sgx', 'dy', 'dyudyn', 'dyudyn_sgy', 'dz', 'dzudzn', 'dzudzn_sgz', 'k', 'k_max', 'kx_max', 'kx_vec', 'ky_max', 'ky_vec', 'kz_max', 'kz_vec', 'nonuniform', 'xn_vec', 'xn_vec_sgx', 'yn_vec', 'yn_vec_sgy', 'zn_vec', 'zn_vec_sgz']
999999999.9999999
keys: ['#refs#', '#subsystem#', 'kgrid', 'sensor_data']
['#refs#', '#subsystem#', 'kgrid', 'sensor_data']
(100001,)
Processed data and metadata saved to: /home/smatsubara/documents/airlift/data/simulation/processed/solid_liquid2_processed.npz


'/home/smatsubara/documents/airlift/data/simulation/processed/solid_liquid2_processed.npz'

### 保存形式のチェック
実機・シミュレーションそれぞれにおいて、変換が正しく実行されたのかを確認します。変換されたファイルを読み込んで、それぞれのファイルの型を確認しましょう。

In [8]:
sim_file_processed="/home/smatsubara/documents/airlift/data/simulation/processed/solid_liquid2_processed.npz"
exp_file_processed="/home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz"

import numpy as np
with np.load(exp_file_processed) as data:
    #print(data.keys())
    print("======== experiment =========")
    print(data['original_keys'])
    print(f"(n_pulses, n_samples, n_channels): {data['processed_data'].shape}")
    print(data['fs'])


with np.load(sim_file_processed) as data:
    #print(data.keys())
    print("======== simulation =========")
    print(data['original_keys'])
    print(f"(n_pulses, n_samples, n_channels): {data['processed_data'].shape}")
    print(data['fs'])
    



======== experiment =========
['__header__' '__version__' '__globals__' 'Tstart' 'Tinterval'
 'ExtraSamples' 'RequestedLength' 'Length' 'Version' 'TDX1' 'TDX2' 'TDX3'
 'TDX1_enlarged']
(n_pulses, n_samples, n_channels): (14988, 5208, 4)
52083333.842615336
======== simulation =========
['#refs#' '#subsystem#' 'kgrid' 'sensor_data']
(n_pulses, n_samples, n_channels): (1, 100001, 1)
999999999.9999999


## 信号波形の可視化・解析
機械学習および信号波形の比較・解析をしやすいように変換した`.npz`ファイルを読み込み、可視化します。これら信号波形は画像形式に変換し解釈の精度を上げることもできます。一方で、一つのパルスの可視化にも意味があります。  
そこで、オプションに従いその画像化を行うスクリプトを`npz2png.py`という関数にまとめてあります。これらの引数として`full=True`ならば画像を、`full=False`ならば一パルス分の概形のみを返します。  
## npz2png (src/util.py)
### input 
 - file_path
 - save_path
 - channel_index
 - start_time
 - end_time
 - full(=True or False)
 - pulse_index  
### output
 - *_"pulse or img".png

In [9]:
from src.utils import npz2png
# npz2img関数を実際に使って画像を保存してみる例
npz_file_path = "/home/smatsubara/documents/airlift/data/experiments/processed/P20241007-1013_processed.npz"
npz_file_path_sim = "/home/smatsubara/documents/airlift/data/simulation/processed/solid_liquid2_processed.npz"
output_folder_path = "/home/smatsubara/documents/airlift/data/visualize"



npz2png(npz_file_path, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=True, pulse_index=0)
npz2png(npz_file_path, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=False, pulse_index=150)
#npz2png(npz_file_path, output_folder_path, channel_index=1, start_time=0.0, end_time=None, full=True, pulse_index=0)
npz2png(npz_file_path_sim, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=True, pulse_index=0)
npz2png(npz_file_path_sim, output_folder_path, channel_index=0, start_time=0.0, end_time=None, full=False, pulse_index=0)

(14988, 5208, 4)
device: cuda
/home/smatsubara/documents/airlift/data/visualize/P20241007-1013_processed_0img.png
(14988, 5208, 4)
/home/smatsubara/documents/airlift/data/visualize/P20241007-1013_processed_0pulse.png
(1, 100001, 1)
device: cuda
/home/smatsubara/documents/airlift/data/visualize/solid_liquid2_processed_0img.png
(1, 100001, 1)
/home/smatsubara/documents/airlift/data/visualize/solid_liquid2_processed_0pulse.png


### 補足
 - 処理に関して  
   生信号を画像化しても意味がないため、画像化の際はヒルベルト変換を実施
 - 前処理  
    飽和を起こしている領域の信号の値を０に（エラーハンドリングを目的とする。）
    データ転送は複数パルスの画像化の際のみ実行
 - 例外処理  
      シミュレーションにおいては０回目以外の測定の値を指定してもエラーを出力　　
      シミュレーション信号の画像生成に関しては、縦１ピクセルの画像と解釈
 - 備考  
      ０回目の測定結果の信号波形の可視化は、ややずれていることが多い

In [4]:
from src import mat2npz_exp,npz2png
import os

import glob

# Get all .mat files in the rawsignal directory
output_dir = "/home/smatsubara/documents/airlift/data/sandbox/experiments/processed"
rawsignal_dir = "/home/smatsubara/documents/airlift/data/sandbox/experiments/rawsignal"
mat_files = glob.glob(os.path.join(rawsignal_dir, "*.mat"))


for file_path in mat_files:
    print(f"Processing {file_path}")
    mat2npz_exp(
        file_path=file_path,
        output_dir=output_dir,
        start_time=0.1,  #初期の信号は不安定であることが多いため除外
        duration=5.0,
        amplitude_threshold=2,
        window_width=0.1e-3,
        signal_key="TDX1"
    )


Processing /home/smatsubara/documents/airlift/data/sandbox/experiments/rawsignal/P20240726-1607.mat
Loading data...
Loading successful
Using device: cuda
Number of detected triggers: (14700,)
arranged_pulses.shape: (14700, 5208, 4)
convert_exp finished
max: inf
processed_data.shape: (14000, 2500, 4)
max: inf
processed_data[0,:,0].shape: (2500,)
max: 0.927592933177948
argmax: 326
maxes argmax: 3,max: 3.4028234663852886e+38
(14000, 1, 4) 0.8806262 3.4028235e+38
scaled: ((14000, 2500, 4), -3.4028235e+38, 3.4028235e+38)
max_value: 3.4028234663852886e+38
['__header__', '__version__', '__globals__', 'Tstart', 'Tinterval', 'ExtraSamples', 'RequestedLength', 'Length', 'Version', 'TDX1', 'TDX2', 'TDX3', 'TDX1_enlarged']
signal points: (2500,)
Processed data and metadata saved to: /home/smatsubara/documents/airlift/data/sandbox/experiments/processed/P20240726-1607_processed.npz
Processing /home/smatsubara/documents/airlift/data/sandbox/experiments/rawsignal/P20240805-1013.mat
Loading data...
Loa

In [7]:
# (H, W, C) -> (C, H, W) への変換
# 例えば processed_data という変数が (H, W, C) であれば以下のようにします

# processed_data = data['processed_data'] という前提で例示
if len(data['processed_data'].shape) == 3:
    chw_data = np.transpose(data['processed_data'], (2, 0, 1))
    print("変換後 shape:", chw_data.shape)
else:
    print('データのshapeが3次元ではありません:', data['processed_data'].shape)


変換後 shape: (4, 14000, 2500)
(14000, 2500, 4)


In [14]:
file_path1="/home/smatsubara/documents/airlift/data/sandbox/experiments/processed/P20240726-1607_processed.npz"
file_path2="/home/smatsubara/documents/airlift/data/sandbox/experiments/processed/P20240924-1525_processed.npz"

data1=np.load(file_path1)
data2=np.load(file_path2)

data1['processed_data'].shape
data2['processed_data'].shape

data1_trans=np.transpose(data1['processed_data'],(2,0,1))
data2_trans=np.transpose(data2['processed_data'],(2,0,1))

print(data1_trans.shape)
print(data2_trans.shape)


# (C, H, W)が2つあるので、(2, C, H, W)にし、(2, 4, 15000, 2500)となるようreshape
combined_data = np.stack([data1_trans, data2_trans], axis=0)
print(f"shape:", combined_data.shape)

# 必要に応じて他の軸(axis=1やaxis=2)で結合も可能ですが、一般的にはチャンネル方向(C)で結合するのが多いです




(4, 14000, 2500)
(4, 14000, 2500)
shape: (2, 4, 14000, 2500)


In [5]:
import polars as pl

df =pl.read_csv("/home/smatsubara/documents/airlift/data/sandbox/experiments/target_valiables.csv",encoding="shift_jis")

df.head()




日付,時分,固相見かけ流速,気相見かけ流速,液相見かけ流速,固相体積率,気相体積率,液相体積率,ガラス球直径
i64,i64,f64,f64,f64,f64,f64,f64,str
20240726,1022,0.0,31.922755,6.059601,0.0,0.749158,0.250842,"""-"""
20240726,1055,0.0,32.685636,6.101101,0.0,0.745521,0.254479,"""-"""
20240726,1113,0.0,32.048131,4.543445,0.0,0.7546132,0.2453868,"""-"""
20240726,1122,0.0,31.968982,4.555286,0.0,0.742794,0.257206,"""-"""
20240726,1334,0.0,32.881976,3.119066,0.0,0.74734,0.25266,"""-"""


In [16]:
data_path = "/home/smatsubara/documents/airlift/data/experiments/processed/solid_liquid"
# "ガラス球直径"列の右隣に、"P2024{日付}{時分}_processed.npz"という名前列を追加
# コピーを作成し、result_dirに保存

# まず既存のdfをコピー
df_copy = df.clone()

# 新しいカラム名
new_col_name = "ファイル名"

# "P2024" + 日付 + 時分 + "_processed.npz"を作成
df_copy = df_copy.with_columns(
    (pl.lit("P") + df_copy["日付"].cast(pl.Utf8) + "-" + df_copy["時分"].cast(pl.Utf8) + "_processed.npz").alias(new_col_name)
)

# "ガラス球直径"列のすぐ右隣りに挿入
glass_col_idx = df_copy.columns.index("ガラス球直径")
cols = df_copy.columns.copy()
cols.insert(glass_col_idx + 1, cols.pop(-1))  # 新しいカラム(最後にある)を目的位置へ

df_final = df_copy.select(cols)

# 保存
result_dir = "/home/smatsubara/documents/airlift/data/sandbox/results" 
save_path = result_dir.rstrip("/") + "/target_valiables_with_filename.csv"
df_final.write_csv(save_path)

print(f"saved: {save_path}")


saved: /home/smatsubara/documents/airlift/data/sandbox/results/target_valiables_with_filename.csv


In [17]:
# "ファイル名" の右隣に data_path と結合したフルパス列 "FullPath" を追加

# 再読込（または前の df_final を使用）
df_work = df_final.clone()

# "FullPath" を作成
fullpath_col = "FullPath"
df_work = df_work.with_columns(
    (pl.lit(data_path.rstrip("/") + "/") + df_work["ファイル名"]).alias(fullpath_col)
)

# "ガラス球直径" のインデックス取得し、2つ右隣に "FullPath" を移動
glass_idx = df_work.columns.index("ガラス球直径")
# "ファイル名"は1つ右、その隣=2つ右になるよう "FullPath" を移動
cols = df_work.columns.copy()
# 一旦 "FullPath" をpop
cols.pop(cols.index(fullpath_col))
cols.insert(glass_idx + 2, fullpath_col)
df_final2 = df_work.select(cols)

# 保存
save_path2 = result_dir.rstrip("/") + "/target_valiables_with_fullpath.csv"
df_final2.write_csv(save_path2)
print(f"saved: {save_path2}")


saved: /home/smatsubara/documents/airlift/data/sandbox/results/target_valiables_with_fullpath.csv


In [18]:
import numpy as np

# "FullPath" 列の各ファイルを np.load でロードし 'x_train_real' 配列だけでなく
# 見かけ流速・体積率6項目（固相見かけ流速, 気相見かけ流速, 液相見かけ流速, 固相体積率, 気相体積率, 液相体積率）
# も行ごとに配列化し、それぞれ np.stack でまとめる
import numpy as np

file_paths = df_final2["FullPath"].to_list()
processed_data_list = []
targets_list = []
for i, p in enumerate(file_paths):
    try:
        data_npz = np.load(p)
    except Exception as e:
        print(f"failed to load: {p} ({e})")
        processed_data_list.append(None)
        continue

    if 'processed_data' not in data_npz:
        print(f"'processed_data' is not in {p}")
        processed_data_list.append(None)
        continue

    arr = data_npz['processed_data']
    print(f"{p}: original shape: {arr.shape}")
    # 軸が3つ（例:(4,15000,2500)）なら、axisを(2,0,1)へ
    if arr.ndim == 3:
        arr_T = np.transpose(arr, (2, 0, 1))
        print(f"    transposed shape: {arr_T.shape}")
        processed_data_list.append(arr_T)
        # 固相見かけ流速, 気相見かけ流速, 液相見かけ流速, 固相体積率, 気相体積率, 液相体積率をtargetとしてappend
        # 元データはdf_final2内にある。i番目のデータのtargetsをまとめる
        # dtypeなどに注意してfloatで取る
        try:
            # .ilocはpolarsでは使えないため、polarsからpandasに変換して取得する（またはrowオブジェクトから直接取得）
            # まずrowで取得し、dictなので列名で値を取り出す
            row = df_final2.row(i)
            targets = np.array([
                float(row[df_final2.columns.index("固相見かけ流速")]),
                float(row[df_final2.columns.index("気相見かけ流速")]),
                float(row[df_final2.columns.index("液相見かけ流速")]),
                float(row[df_final2.columns.index("固相体積率")]),
                float(row[df_final2.columns.index("気相体積率")]),
                float(row[df_final2.columns.index("液相体積率")])
            ])
            targets_list.append(targets)
        except Exception as e:
            print(f"failed to load targets for {p} ({e})")
            processed_data_list.append(None)
            continue
        
    else:
        print(f"    processed_data is not 3D (got shape {arr.shape}), skipping")
        processed_data_list.append(None)

# Noneを取り除き、すべて有効ならstack
processed_data_valids = [a for a in processed_data_list if a is not None]
if len(processed_data_valids) == 0:
    print("No valid data could be loaded. combined_data will be None")
    combined_data = None
else:
    combined_data = np.stack(processed_data_valids, axis=0)
    print(f"shape: {combined_data.shape}")

# 次々に結合したい場合、file_pathsをdf_final2["FullPath"]など全件で行えば良い
# ---
# 必要に応じて結合軸(axis=0=バッチ、axis=1=チャンネル結合等)を調整
x_train_real = combined_data
t_train_real = np.stack(targets_list, axis=0)

print(x_train_real.shape)
print(t_train_real.shape)

np.savez("/home/smatsubara/documents/airlift/data/sandbox/results/x_train_real.npz", x_train_real=x_train_real)
np.savez("/home/smatsubara/documents/airlift/data/sandbox/results/t_train_real.npz", t_train_real=t_train_real)


failed to load: /home/smatsubara/documents/airlift/data/experiments/processed/solid_liquid/P20240726-1022_processed.npz ([Errno 2] No such file or directory: '/home/smatsubara/documents/airlift/data/experiments/processed/solid_liquid/P20240726-1022_processed.npz')
failed to load: /home/smatsubara/documents/airlift/data/experiments/processed/solid_liquid/P20240726-1055_processed.npz ([Errno 2] No such file or directory: '/home/smatsubara/documents/airlift/data/experiments/processed/solid_liquid/P20240726-1055_processed.npz')
failed to load: /home/smatsubara/documents/airlift/data/experiments/processed/solid_liquid/P20240726-1113_processed.npz ([Errno 2] No such file or directory: '/home/smatsubara/documents/airlift/data/experiments/processed/solid_liquid/P20240726-1113_processed.npz')
failed to load: /home/smatsubara/documents/airlift/data/experiments/processed/solid_liquid/P20240726-1122_processed.npz ([Errno 2] No such file or directory: '/home/smatsubara/documents/airlift/data/experi

In [23]:
# x_train_realのshapeは(バッチ数, チャンネル, 14000, 2500)なので、
# メモリエラー回避のため、(N, C, 14000, 2500)→(N, C, 1400, 2500)のダウンサンプリングはサンプル・チャンネルごと（バッチ＆チャンネルごと）にブロック処理で実施する
import numpy as np
from scipy.signal import resample

if x_train_real.shape[2] == 14000:
    N, C, T, S = x_train_real.shape
    x_downsampled = np.empty((N, C, 1400, S), dtype=x_train_real.dtype)
    for n in range(N):
        for c in range(C):
            # 1サンプル1チャンネルずつ(14000, 2500)→(1400, 2500)にメモリ節約しつつresample
            x_downsampled[n, c] = resample(x_train_real[n, c], 1400, axis=0)
        print(f"  Downsampled batch {n+1}/{N}", flush=True)
    x_train_real = x_downsampled
    print(f"Downsampled x_train_real to shape: {x_train_real.shape}")
else:
    print(f"x_train_real shape unexpected for downsampling: {x_train_real.shape}")


  Downsampled batch 1/108


/home/smatsubara/documents/sandbox/.venv/lib/python3.8/site-packages/scipy/signal/_signaltools.py:3165: RuntimeWarning: overflow encountered in multiply
  Y[tuple(sl)] *= 2.
/home/smatsubara/documents/sandbox/.venv/lib/python3.8/site-packages/scipy/signal/_signaltools.py:3165: RuntimeWarning: invalid value encountered in multiply
  Y[tuple(sl)] *= 2.


  Downsampled batch 2/108
  Downsampled batch 3/108
  Downsampled batch 4/108
  Downsampled batch 5/108
  Downsampled batch 6/108
  Downsampled batch 7/108
  Downsampled batch 8/108
  Downsampled batch 9/108
  Downsampled batch 10/108
  Downsampled batch 11/108
  Downsampled batch 12/108
  Downsampled batch 13/108
  Downsampled batch 14/108
  Downsampled batch 15/108
  Downsampled batch 16/108
  Downsampled batch 17/108
  Downsampled batch 18/108
  Downsampled batch 19/108
  Downsampled batch 20/108
  Downsampled batch 21/108
  Downsampled batch 22/108
  Downsampled batch 23/108
  Downsampled batch 24/108
  Downsampled batch 25/108
  Downsampled batch 26/108
  Downsampled batch 27/108
  Downsampled batch 28/108
  Downsampled batch 29/108
  Downsampled batch 30/108
  Downsampled batch 31/108
  Downsampled batch 32/108
  Downsampled batch 33/108
  Downsampled batch 34/108
  Downsampled batch 35/108
  Downsampled batch 36/108
  Downsampled batch 37/108
  Downsampled batch 38/108
  Downsam

In [24]:
np.save("/home/smatsubara/documents/airlift/data/sandbox/results/x_train_real.npy", x_train_real)
np.save ("/home/smatsubara/documents/airlift/data/sandbox/results/t_train_real.npy", t_train_real)

In [25]:
import numpy as np

x_train_real = np.load("/home/smatsubara/documents/airlift/data/sandbox/results/x_train_real.npy")
t_train_real = np.load("/home/smatsubara/documents/airlift/data/sandbox/results/t_train_real.npy")

print(x_train_real.shape)
print(t_train_real.shape)

(108, 4, 1400, 2500)
(108, 6)
